In [ ]:
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go

import plotly.figure_factory as ff
import numpy as np

import os
import warnings

In [ ]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching

In [ ]:
WRITE_TO_FILE = False

In [ ]:
def default_styling(fig):
    
    fig.update_layout(
        font=dict(
            family="Times New Roman",
            size=20,      
            ),
        legend=dict(
            yanchor="top",
            y=0.24,
            xanchor="right",
            x=1,
            bgcolor="white",
            bordercolor="Black",
            borderwidth=1
        )
    )
    
    fig.update_layout(uniformtext_minsize=28, uniformtext_mode='hide',font=dict(family="Times New Roman"))
    

    fig.update_xaxes(showline=True, showgrid=True, gridwidth=0.1, 
                     gridcolor='rgba(1,1,1,0.15)', 
                     linecolor='black',
                    zerolinecolor='rgba(1,1,1,0.25)')
    fig.update_yaxes(showline=True, showgrid=True, gridwidth=0.1, 
                     gridcolor='rgba(1,1,1,0.15)', 
                     linecolor='black',
                     zerolinecolor='rgba(1,1,1,0.25)')

    fig.update_layout(
        margin=dict(l=20, r=30, t=20, b=0),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        width=750,
        height=500,
    )    
    
#     fig.show()
#     fig.write_image(f"images/{key}.pdf")

In [ ]:
os.listdir("tables")

In [ ]:
dataset = "wikitext"
obj = "causal_lang_training"

In [ ]:
summary_df = pd.read_csv(f'tables/table_BENCHMARK_breach_{obj}_{dataset}_reports.csv', sep='\t')

In [ ]:
model = "transformer3"
metric = "accuracy" # "max-sentence_accuracy"
seq_length = 32

In [ ]:
summary_df.keys()

In [ ]:
df = (
    summary_df[(summary_df.model==model) & (summary_df.name.str.contains(f"_{seq_length}_"))]
    .groupby(["name", "datapoints"], as_index=False)[metric]
    .agg(["mean"])
    .reset_index()
)
df

In [ ]:
experiments = ["UNDIV", "S1", "S25", "S2", "S3", "SBBF", "TFirst", "tokenreplace", 
              "noEXT_trafo3_baseparams", "baseparams", "noEXT"]

In [ ]:
def simplify_name(name):
    for exp in experiments:
        if exp in name:
            return exp
    return "base"

In [ ]:
df["experiment"] = df["name"].map(simplify_name)
df = df.sort_values("datapoints")
df

In [ ]:
fig = px.scatter(df, x="datapoints", y="mean", title='Hyperparameter Variations', color="experiment",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=3, dash='solid'))


# default_styling(fig)
# fig.update_xaxes(type="log")
# fig.update_layout(legend=dict(y=0.39, x=1))
fig.show()

# Threshold Sensitivity

In [ ]:
df_t = df[df.experiment.isin(["S1", "S25", "S2", "base"])]

df_t = df_t.replace({
                        'S1':"sd=1", 
                        'base':"sd=1.5", 
                        'S2':"sd=2", 
                        'S25':"sd=2.5", 
                        }
                         )

fig = px.scatter(df_t, x="datapoints", y="mean", color="experiment",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)
#fig.update_xaxes(type="log")
fig.update_xaxes(range=[0, 64])
fig.update_layout(legend=dict(y=0.39, x=0.257))
fig.show()

# No External Data

In [ ]:
df_ext = df[df.experiment.isin(["noEXT", "base"])]

df_ext = df_ext.replace({
                        'base':"Estimated on public Data", 
                        'noEXT':"Estimated on random data", 
                        }
                         )

fig = px.scatter(df_ext, x="datapoints", y="mean", color="experiment",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))
#fig.data[1].line.dash="dash"


default_styling(fig)
#fig.update_xaxes(range=[0, 64])
fig.update_layout(legend=dict(y=0.25, x=0.4))
fig.show()

# Dataset variations:

In [ ]:
obj = "causal_lang_training"
datasets = ["shakespeare", "stackoverflow", "wikitext", "random-tokens"]
dfs = []
for dataset in datasets:
    dfs.append(pd.read_csv(f'tables/table_BENCHMARK_breach_{obj}_{dataset}_reports.csv', sep='\t'))
summary_df = pd.concat(dfs, keys=datasets).reset_index()

In [ ]:
model = "transformer3"
metric = "accuracy"
seq_length = 32

In [ ]:
df = (
    summary_df[(summary_df.model==model) & (summary_df.name.str.contains(f"_{seq_length}_"))]
    .groupby(["name", "datapoints", "level_0"], as_index=False)[metric]
    .agg(["mean"])
    .reset_index()
)
df["experiment"] = df["name"].map(simplify_name)
df = df[df.experiment.isin(["base"])].sort_values("datapoints")
df["is_base"] = df.experiment == "base"

In [ ]:
fig = px.scatter(df, x="datapoints", y="mean", color="level_0", #symbol="is_base",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
                "level_0" : "Dataset",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)
#fig.update_xaxes(type="log")
fig.update_layout(legend=dict(y=0.39, x=0.257))
fig.show()

# Print recovered total tokens

In [ ]:
dataset = "wikitext"
obj = "causal_lang_training"

In [ ]:
summary_df = pd.read_csv(f'tables/table_BENCHMARK_breach_{obj}_{dataset}_reports.csv', sep='\t')

In [ ]:
model = "transformer3"
metric = "accuracy" # "max-sentence_accuracy"
seq_length = 32

In [ ]:
df = (
    summary_df[(summary_df.model==model) & (summary_df.name.str.contains(f"_{seq_length}_"))]
    .groupby(["name", "datapoints"], as_index=False)[metric]
    .agg(["mean"])
    .reset_index()
)
df["experiment"] = df["name"].map(simplify_name)
df = df.sort_values("datapoints")
df

In [ ]:
df["tokens_recovered"] = df["datapoints"] * 32 * df["mean"]

In [ ]:
df_ext = df[df.experiment.isin(["base", "baseparams"])]

df_ext = df_ext.replace({
                        'base':"Estimated on public Data", 
                        'noEXT':"Estimated on random data", 
                        }
                         )

fig = px.scatter(df_ext, x="datapoints", y="tokens_recovered", color="experiment",
                                 labels={ # replaces default labels by column name
                "experiment": "Experiment",  
                "mean": f"Total Accuracy", 
                "real_size": "Real Dataset Samples",
                "datapoints": "Batch Size",
            })
fig.update_traces(mode='lines+markers', line=dict(width=5, dash='solid'))


default_styling(fig)
fig.update_xaxes(range=[0, 1024])
fig.update_layout(legend=dict(y=0.25, x=0.4))
fig.show()